In [3]:
import numpy as np
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', 100)  

#Beautiful Soup and Requests
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

#Import Regex
import re

#URLIB
import urllib

In [4]:
def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

# Scrape Auction Links

In [5]:
run = True
if run == True:
    url = 'https://www.bestnetleiloes.com/pt/leiloes/terminados'
    dict_url_links = {}
    flag = True
    s = requests.Session()
    list_links_auctions = set()
    counter = 0
    #Loop over all pages
    while flag == True:
        #Get html and links of contemporary art auctions
        soup = BeautifulSoup(requests_retry_session(session=s).get(url).content)
        tmp_list_links_auctions = soup.find_all('a', href=True, title='ARTE CONTEMPORÂNEA')
        #Drop duplicates by converting to set
        tmp_list_links_auctions = set([i['href'] for i in tmp_list_links_auctions])
        #Join the set on the list of url
        list_links_auctions = list_links_auctions.union(tmp_list_links_auctions)
        #Get next page
        try:
            url = soup.find_all('a', href=True, text='Página Seguinte')[0]['href']
        except:
            print('Final page reached!')
            flag=False
        counter += 1    
        if counter % 25 == 0:
            print('{} pages have been scraped.'.format(counter))

    #Clean the list to ensure only full auctions remain
    list_url_auctions = [i for i in list_links_auctions if 'contemporanea-' in i]
    #Convert to pandas DataFrame
    df_auction_links = pd.DataFrame(list_url_auctions,columns=['URL'])
    #Sort the auctions before saving 
    df_auction_links['Auction Number'] = df_auction_links['URL'].str.split('-').apply(lambda x: int(x[-1]))
    df_auction_links = df_auction_links.sort_values('Auction Number',ascending=False)

    #Load previous auction_links
    df_auction_links_previous = pd.read_csv('auction_links.csv',index_col=0)

    to_run= True
    #If we want to run analysis only on the new auctions, run the following if and get the dataframe of only the new auctions
    if to_run==True:
        dfs_dictionary = {'DF1':df_auction_links,'DF2':df_auction_links_previous}
        df = pd.concat(dfs_dictionary)
        df = df.drop_duplicates(keep=False).copy()

    #Save to file
    df_auction_links.to_csv('auction_links_new.csv')
    print('All auction links were scraped and saved to file!')

25 pages have been scraped.
50 pages have been scraped.
75 pages have been scraped.
100 pages have been scraped.
125 pages have been scraped.
150 pages have been scraped.
175 pages have been scraped.
200 pages have been scraped.
225 pages have been scraped.
250 pages have been scraped.
Final page reached!
All auction links were scraped and saved to file!


# Scrape Artwork Links

In [6]:
run = True
if run == True:
    list_artwork_links = []
    list_auction_dates = []
    counter = 0

    for i in df['URL'].tolist():
        #Open an Auction Page
        url = i
        soup = BeautifulSoup(requests_retry_session(session=s).get(url).content)

        tmp_list_artwork_links = soup.find_all('div', attrs={'class':'content'})
        list_artwork_links = list_artwork_links + [i.find_all('a', href=True)[0]['href'] for i in tmp_list_artwork_links]

        #Store Dates
        tmp_date = soup.find_all('div',attrs={'class':'info'})[0].text.split('\n')[-2]
        tmp_list_date = [tmp_date]*len([i.find_all('a', href=True)[0]['href'] for i in tmp_list_artwork_links])
        list_auction_dates = list_auction_dates + tmp_list_date
        if counter %25 == 0:
            print('Scraped artwork links from {} auctions.'.format(counter))
        counter +=1

    #Save the URL list to a file
    df_artwork_links = pd.DataFrame({'URL':list_artwork_links,'Date':list_auction_dates},columns=['Date','URL'])
    #Read old artwork links from file
    df_artwork_links_previous = pd.read_csv('artwork_links.csv')
    df_artwork_links_full = pd.concat((df_artwork_links,df_artwork_links_previous))
    #Store artwork links on file
    df_artwork_links_full.to_csv('artwork_links.csv')

Scraped artwork links from 0 auctions.


# Scrape Artworks

In [29]:
dic_artworks = {'Auction Number':[],
               'Number of Bids':[],
               'Author':[],
               'Title':[],
               'Technique':[],
                'Is Signed':[],
               'Dimensions':[],
               'All Characteristics':[],
               'Sale Price':[],
               'Image URL':[],
                'Date of Auction End':[],
                'Lot':[]
}
counter = 0

for artwork_nbr in range(df_artwork_links.shape[0]):
    #Get Date
    tmp_date = df_artwork_links['Date'].iloc[artwork_nbr]

    #Read HTML
    url = df_artwork_links['URL'].iloc[artwork_nbr]
    soup = BeautifulSoup(requests_retry_session(session=s).get(url).content)

    #-------------------------------------------- Scrape information ---------------------------------------
    #Auction Number
    try:
        tmp_auction_nbr = url.split('/')[-2]
        dic_artworks.setdefault('Auction Number', []).append(tmp_auction_nbr)
    except:
        tmp_auction_nbr = np.nan
        dic_artworks.setdefault('Auction Number', []).append(tmp_auction_nbr)
    #Number of bids
    try:
        tmp_nbr_bids = int(soup.find_all('div',attrs={'class':'info'})[1].text.split('Licitações: ')[-1].rstrip('\n\n'))
        dic_artworks.setdefault('Number of Bids', []).append(tmp_nbr_bids)
    except:
        tmp_nbr_bids = np.nan
        dic_artworks.setdefault('Number of Bids', []).append(tmp_nbr_bids)    
    #Artist
    try:
        tmp_author = soup.find_all('div',attrs={'class':'description'})[0].text.split('\n')[1]
        dic_artworks.setdefault('Author', []).append(tmp_author)
    except:
        tmp_author = np.nan
        dic_artworks.setdefault('Author', []).append(tmp_author) 
    #Title
    try:
        tmp_title = soup.find_all('div',attrs={'class':'description'})[0].text.split('\n')[2]
        dic_artworks.setdefault('Title', []).append(tmp_title) 
    except:
        tmp_title = np.nan
        dic_artworks.setdefault('Title', []).append(tmp_title) 
    #Technique
    try:
        tmp_technique = soup.find_all('div',attrs={'class':'description'})[0].text.split('\n')[3]
        dic_artworks.setdefault('Technique', []).append(tmp_technique) 
    except:
        tmp_technique = np.nan
        dic_artworks.setdefault('Technique', []).append(tmp_technique) 
    #Check if it signed
    try:
        tmp_is_signed = soup.find_all('div',attrs={'class':'description'})[0].text.split('\n')[4]
        dic_artworks.setdefault('Is Signed', []).append(tmp_is_signed) 
    except:
        tmp_is_signed = np.nan
        dic_artworks.setdefault('Is Signed', []).append(tmp_is_signed)     
    #Dimensions
    try:
        tmp_dimension = soup.find_all('div',attrs={'class':'description'})[0].text.split('\n')[5]
        dic_artworks.setdefault('Dimensions', []).append(tmp_dimension)     
    except:
        tmp_dimension = np.nan
        dic_artworks.setdefault('Dimensions', []).append(tmp_dimension)      
    #Store all characteristics (for future analysis of additional or missing information)
    try:
        tmp_string_characteristics = soup.find_all('div',attrs={'class':'description'})[0].text
        dic_artworks.setdefault('All Characteristics', []).append(tmp_string_characteristics)  
    except:
        tmp_string_characteristics = np.nan
        dic_artworks.setdefault('All Characteristics', []).append(tmp_string_characteristics)
    #Sale Price
    try:
        tmp_sell_price = int(soup.find_all('div',attrs={'class':'sell-price'})[0].text.split('por ')[-1].split('EUR')[0].strip().split('.')[0])
        dic_artworks.setdefault('Sale Price', []).append(tmp_sell_price)
    except:
        tmp_sell_price = np.nan
        dic_artworks.setdefault('Sale Price', []).append(tmp_sell_price)
    #Image URL's
    try:
        tmp_image_url = soup.find_all('a', href=True,attrs={'class','colorbox'})[0]['href']
        dic_artworks.setdefault('Image URL', []).append(tmp_image_url)
    except:
        tmp_image_url = np.nan
        dic_artworks.setdefault('Image URL', []).append(tmp_image_url)

    dic_artworks.setdefault('Date of Auction End', []).append(tmp_date)
    tmp_lot = soup.find_all('div',attrs={'class':'info'})[1].text.split('Lote: ')[1].split('\n')[0]
    dic_artworks.setdefault('Lot', []).append(tmp_lot)
    #Save Image to folder
    try:
        urllib.request.urlretrieve(tmp_image_url, 'artwork_images/' + tmp_auction_nbr +'_'+tmp_lot+'.jpg')
    except:
        print('Um erro foi encontrado')
    
    counter += 1
    if counter %100 == 0:
        print('{} artworks scraped!'.format(counter))

#Create DataFrame
df_artworks = pd.DataFrame(dic_artworks)

#Get previously scraped artworks
df_previous_artworks = pd.read_csv('artworks.csv',encoding='utf-16',index_col=0)

#Concatenate DataFrames
df_final = pd.concat((df_artworks,df_previous_artworks),axis=0)
#Save CSV
df_final.to_csv('artworks.csv',encoding='utf-16')

100 artworks scraped!
200 artworks scraped!
300 artworks scraped!


In [26]:
df_artworks.shape

(357, 12)

In [25]:
df_previous_artworks.shape

(5, 12)

In [27]:
df_previous_artworks

,Auction Number,Number of Bids,Author,Title,Technique,Is Signed,Dimensions,All Characteristics,Sale Price,Image URL,Date of Auction End,Lot
0,arte-contemporanea-593,52,Noronha da Costa,Sem título,Tinta celulósica sobre tela,Assinada,40x50 cm,\nNoronha da Costa\nSem título\nTinta celulósi...,940.0,https://www.bestnetleiloes.com/media/lots/7426...,27 de Março a 7 de Abril de 2020,69589
1,arte-contemporanea-593,15,Bual,Retrato (César Monteiro?),Serigrafia VII/XVII,Assinada e datada 97,70x50 cm - dim.total,\nBual\nRetrato (César Monteiro?)\nSerigrafia ...,110.0,https://www.bestnetleiloes.com/media/lots/7426...,27 de Março a 7 de Abril de 2020,68343
2,arte-contemporanea-593,12,Luzia Lage,Sem título,Serigrafia intervencionada 21/100,Assinada,"40x51,5 cm - a mancha / 51x72 cm - dim.total",\nLuzia Lage\nSem título\nSerigrafia intervenc...,65.0,https://www.bestnetleiloes.com/media/lots/7426...,27 de Março a 7 de Abril de 2020,68228
3,arte-contemporanea-593,22,David Levy Lima,Lisboa,Óleo sobre tela,Assinado,20x50 cm,\nDavid Levy Lima\nLisboa\nÓleo sobre tela\nAs...,210.0,https://www.bestnetleiloes.com/media/lots/7426...,27 de Março a 7 de Abril de 2020,69590
4,arte-contemporanea-593,33,Alfredo Luz,Sem título,Óleo sobre tela,Assinado,60x50 cm,\nAlfredo Luz\nSem título\nÓleo sobre tela\nAs...,570.0,https://www.bestnetleiloes.com/media/lots/7426...,27 de Março a 7 de Abril de 2020,67337
